## Laboratory work 3
Determining and removing drawbacks of exponential and running mean

Team: Abramov Semen, Belikov Ilia, Korobkov Roman

In [20]:
import pandas as pd
import numpy as np
from math import pi
from sklearn.metrics import mean_squared_error

In [21]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [22]:
def generate_randaccel_trajectory(X0=5, V0=0, var=10, size=300, T=0.1):
    V = np.zeros(size)
    V[0] = V0
    
    a = np.random.normal(scale=np.sqrt(var), size=size)
    
    V[1:] = a[:-1]*T 
    V = np.cumsum(V)
    
    X = np.zeros(size)
    X[0] = X0
    X[1:] = V[:-1]*T + a[:-1]*(T**2)/2
    return np.cumsum(X)

In [23]:
def measure(true_traj, var=0.1):
    return np.add(true_traj, np.random.normal(scale=np.sqrt(var), size=true_traj.shape))

In [24]:
def smooth_coefficient(sigm_w, sigm_n):
    Xi = sigm_w/sigm_n
    return (-Xi + np.sqrt(Xi**2 + 4*Xi))/2

In [25]:
def smooth(z, alpha):
    X = np.zeros(z.shape)
    X[0] = z[0]
    for i in range(1, z.shape[0]):
        X[i] = X[i-1] + alpha*(z[i] - X[i-1])
    return X

In [62]:
tr = generate_randaccel_trajectory(X0=5, V0=1, T=1, size=200, var=0.2**2)

In [63]:
msr = measure(true_traj=tr, var=20**2)

In [68]:
def kalman(msr, X0=2.0, P0=10000.0, Q=1e-5, R=20**2):
    size = (len(msr),) # size of array

    # allocate space for arrays
    X=np.zeros(size)
    P=np.zeros(size)
    K=np.zeros(size)
    
    # intial guesses
    X[0] = X0
    P[0] = P0

    for k in range(1, size):

        P_ = P[k-1]+Q
        # measurement update
        K[k] = P_/(P_+R )
        X[k] = X[k-1]+K[k]*(msr[k]-X[k-1])
        P[k] = (1-K[k])*P_

    return X

In [70]:
data = [
    go.Scatter(
        y=tr,
        name='true trajectory'
    ),
    go.Scatter(
        y=msr,
        name='measurement'
    ),
    go.Scatter(
        y=kalman_1(msr, Q=5, R=1000),
        name='kalman'
    ),
]

layout= go.Layout(
    title= 'Exp smoothing VS running mean',
    xaxis= dict(
        title= 'Step',
    ),
    yaxis=dict(
        title= 'X',
    ),
    showlegend= True
)
fig= go.Figure(data=data, layout=layout)
iplot(fig)